# CLASE 2.3: INDEXACIÓN JERÁRQUICA.
---
## Índices multinivel.
Hasta este punto, nos hemos enfocado principalmente en data de tipo uni y bidimensional, almacenada en series y DataFrames de **Pandas**, respectivamente. Con frecuencia, es útil ir más allá y almacenar estructuras de datos de mayor dimensión; es decir, data indexada con más de una o más llaves. Una práctica común para la construcción de estructuras de datos de este tipo resulta en la utilización de un **esquema de indexado jerárquico** (conocido también cono **indexado multinivel**) para incorporar múltiples niveles de indexación en un único índice. De esta manera, la data en mayores dimensiones puede ser representada de forma compacta mediante objetos familiares, tales como las series y DataFrames que hemos manipulado hasta ahora.

En esta sección, exploraremos la creación directa de objetos de tipo `pd.MultiIndex`, consideraciones tales como indexación, slicing y cálculo de estadígrafos sobre data indexada de esta manera, y rutinas muy útiles para la conversión entre representaciones simples y con indexación múltiple de nuestros datos.

Comencemos intentando representar compactamente data de dos dimensiones mediante una serie de **Pandas**. Sin perder generalidad, consideraremos una serie de datos donde cada punto tiene asignado un carácter y una llave numérica.

In [1]:
import numpy as np
import pandas as pd

### Como no debemos hacerlo.
Supongamos que nos gustaría mantener un registro de datos de cada dos años de alimentación diaria para una planta de molienda SAG con tres líneas de procesamiento de mineral. Usando las herramientas de **Pandas** que ya hemos cubierto, podríamos vernos tentados simplemente a utilizar tuplas de Python como llaves:

In [2]:
# Un intento de índice múltiple.
idx = [
    ("SAG 1", 2019), ("SAG 1", 2020),
    ("SAG 2", 2019), ("SAG 2", 2020),
    ("SAG 3", 2019), ("SAG 3", 2020),
]

In [3]:
# Los valores que asociaremos a este índice múltiple.
feed = np.array([
    150350, 156780,
    149187, 151223,
    159003, 157743
])

In [4]:
# Construimos nuestra serie.
feed_ser = pd.Series(data=feed, index=idx)

In [5]:
# Mostramos nuestra serie en pantalla.
feed_ser

(SAG 1, 2019)    150350
(SAG 1, 2020)    156780
(SAG 2, 2019)    149187
(SAG 2, 2020)    151223
(SAG 3, 2019)    159003
(SAG 3, 2020)    157743
dtype: int64

Con este esquema de indexación, podemos generar seleccionar elementos o generar slices directamente:

In [6]:
feed_ser[("SAG 1", 2019):("SAG 2", 2020)]

(SAG 1, 2019)    150350
(SAG 1, 2020)    156780
(SAG 2, 2019)    149187
(SAG 2, 2020)    151223
dtype: int64

Pero lo útil termina ahí, ya que si necesitamos seleccionar valores únicamente del año 2019, necesitaremos realizar trabajo con código bastante aparatoso (y potencialmente de lenta ejecución), rogando para que eventualmente funcione:

In [7]:
feed_ser[[i for i in feed_ser.index if i[1] == 2019]]

(SAG 1, 2019)    150350
(SAG 2, 2019)    149187
(SAG 3, 2019)    159003
dtype: int64

### Como sí debemos hacerlo.
Afortunadamente, **Pandas** nos provee de una mejor forma de lograr esto. Nuestro indexado múltiple basado en tuplas hecho previamente corresponde a una forma rudimentaria de generar un índice multinivel, y el objeto `pd.MultiIndex` de **Pandas** nos provee de una manera sencilla de generar indexaciones múltiples con objetos tales como series y DataFrames. Podemos crear un índice multinivel a partir de tuplas por medio de la clase `pd.MultiIndex`, aplicando inmediatamente el método `from_tuples()`:

In [8]:
# Creación de un índice multinivel a partir de una colección de tuplas.
idx = pd.MultiIndex.from_tuples(idx)
idx

MultiIndex([('SAG 1', 2019),
            ('SAG 1', 2020),
            ('SAG 2', 2019),
            ('SAG 2', 2020),
            ('SAG 3', 2019),
            ('SAG 3', 2020)],
           )

Notemos que este objeto de tipo `pd.MultiIndex` contiene múltiples niveles de indexación. En este caso de ejemplo, los molinos y los años respectivos, así que como múltiples etiquetas que encodean tales niveles.

Si reindexamos nuestra serie con este índice multinivel (mediante el método `reindex()`), veremos la representación jerárquica de nuestra data:

In [9]:
feed_ser = feed_ser.reindex(idx)
feed_ser

SAG 1  2019    150350
       2020    156780
SAG 2  2019    149187
       2020    151223
SAG 3  2019    159003
       2020    157743
dtype: int64

En el ejemplo anterior, las primeras dos columnas de la serie muestran los múltiples valores que permiten la indexación respectiva, mientras que la tercera columna representan la data propiamente tal (que obtendriamos mediante el atributo `values`). Notemos además que hay registros faltantes (pero que no son `nan`) en la primera columna, lo cual es un recurso de los índices multinivel: Cualquier registro en blanco tienen el mismo valor que el inmediatamente superior.

Ahora, para acceder a toda la data cuyo segundo índice sea 2020, podemos utilizar simplemente la notación de indexado de **Pandas**:

In [10]:
feed_ser[:, 2020]

SAG 1    156780
SAG 2    151223
SAG 3    157743
dtype: int64

El resultado es un arreglo con índice simple, y únicamente con las llaves que son de nuestro interés. Este tipo de sintaxis es mucho más conveniente (y las operaciones resultan ser mucho más eficientes) que la sintaxis basada en tuplas construida de la forma rudimentaria presentada en un principio.

## El índice multinivel como dimensión extra.
Podríamos darnos cuenta de algo adicional en el ejemplo anterior: Podríamos haber construido un DataFrame que almacenara la misma data con las respectivas etiquetas en sus columnas. De hecho, **Pandas** está construido con esta equivalencia en consideración. El método `unstack()` (que ya habíamos visto en la sección anterior) es capaz de convertir rápidamente una serie multinivel en un DataFrame convencional:

In [11]:
# Podemos desapilar una serie multinivel con el método unstack().
feed_df = feed_ser.unstack()
feed_df

,2019,2020
SAG 1,150350,156780
SAG 2,149187,151223
SAG 3,159003,157743


Naturalmente, el método `stack()` nos permite lograr lo contrario: Construir una sere multinivel a partir de un DataFrame:

In [12]:
# Podemos apilar un DataFrame en una serie multinivel con el método stack():
feed_df.stack()

SAG 1  2019    150350
       2020    156780
SAG 2  2019    149187
       2020    151223
SAG 3  2019    159003
       2020    157743
dtype: int64

Viendo lo anterior, cabe la posibilidad de preguntarnos.. *¿Por qué molestarnos con todo este tema del indexado jerárquico?* Y la respuesta es muy simple: De la misma forma que hemos sido capaces de utilizar índices múltiples para representar data bidimensional en una serie, podemos extender este procedimiento a dimensiones superiores (por ejemplo, data 4D en un DataFrame). Cada nivel adicional en un índice múltiple representa una dimensión extra de la data; tomar ventaja de esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer adicionar una columna de data relativa a leyes de mineral y recuperación de cobre para cada uno de estos circuitos en cada año; con un índice multinivel, aquello es tan sencillo como agregar otra columna a un DataFrame:

In [13]:
# Construimos un DataFrame usando como base nuestra serie multinivel.
feed_df = pd.DataFrame(
    {
        "tratamiento": feed_ser,
        "ley_cut": np.array([0.56, 0.63, 0.49, 0.51, 0.55, 0.43]),
        "recup_cu": np.array([90.3, 90.8, 91.4, 89.9, 90.3, 92.2])
    }
)

In [14]:
# Mostramos este DataFrame en pantalla.
feed_df

tratamiento  ley_cut  recup_cu
SAG 1 2019       150350     0.56      90.3
      2020       156780     0.63      90.8
SAG 2 2019       149187     0.49      91.4
      2020       151223     0.51      89.9
SAG 3 2019       159003     0.55      90.3
      2020       157743     0.43      92.2

En adición a lo anterior, todas las `ufuncs` estudiadas previamente también funcionan perfectamente con objetos de **Pandas** con indexación multinivel. Por ejemplo, computemos el cobre fino extraído diario para cada uno de estos circuitos:

In [15]:
cuf = feed_df["tratamiento"] * (feed_df["ley_cut"]/100) * (feed_df["recup_cu"]/100)
cuf

SAG 1  2019    760.289880
       2020    896.844312
SAG 2  2019    668.148898
       2020    693.342333
SAG 3  2019    789.688400
       2020    625.387898
dtype: float64

In [16]:
cuf.unstack()

,2019,2020
SAG 1,760.289880,896.844312
SAG 2,668.148898,693.342333
SAG 3,789.688400,625.387898


## Métodos de creación de índices multinivel.
La forma más directa de construir una serie o DataFame multinivel es simplemente pasar una lista de dos o más arreglos al constructor de índices respectivo. Por ejemplo: 

In [17]:
df = pd.DataFrame(
    data=np.random.rand(4, 2),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=['x1', 'x2']
)

In [18]:
df

x1        x2
a 1  0.440159  0.362901
  2  0.303541  0.042880
b 1  0.525629  0.732826
  2  0.895001  0.755538

Evidentemente, el trabajo de crear un objeto de tipo `pd.MultiIndex` en el ejemplo anterior es hecho *tras bambalinas*.

Similarmente, si pasamos un diccionario con tuplas apropiadas como llaves, **Pandas** automáticamente reconocerá dicha estructura y creará un objeto de tipo `pd.MultiIndex` por defecto:

In [19]:
# Construcción directa de una serie multinivel a partir de un diccionario con tuplas como llaves.
data = {
    ("SAG 1", 2019): 150350, 
    ("SAG 1", 2020): 156780,
    ("SAG 2", 2019): 149187, 
    ("SAG 2", 2020): 151223,
    ("SAG 3", 2019): 159003, 
    ("SAG 3", 2020): 157743,
}

In [20]:
pd.Series(data)

SAG 1  2019    150350
       2020    156780
SAG 2  2019    149187
       2020    151223
SAG 3  2019    159003
       2020    157743
dtype: int64

No obstante, a veces es útil crear explícitamente un objeto de tipo `pd.MultiIndex`; veremos algunos ejemplos de esto a continuación.

### Constructores explícitos de índices multinivel.
Para mayor flexibilidad a la hora de construir un índice multinivel, podemos utilizar los métodos constructores de clase disponibles en `pd.MultiIndex`. Por ejemplo, tal y como lo hicimos antes, podemos construir el índice multinivel a partir de una simple lista de arreglos que denotan los índices, explícitamente, en todo nivel:

In [21]:
# Construcción de índice multinivel de forma explicita a partir de un par de arreglos.
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Podemos construirlo a partir de una lista de tuplas que nos dan los múltiples valores de los índices en cada punto:

In [22]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Y también podemos construirlo incluso a partir del producto cartesiano de dos listas, donde la primera de ellas conforma el primer nivel del índice, y así sucesivamente:

In [23]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Cualquiera de los índices multinivel construidos previamente puede ser pasado al argumento `index` del constructor de cualquier serie o DataFrame, o ser pasado al método `reindex` para su correspondiente reindexación.

### Nombres asociados a los niveles de un índice.
A veces es conveniente cambiar los niveles de un índice múltiple. Aquello puede lograrse pasando el argumento `names` a cualquiera de los constructores previamente vistos, o bien, seteando el atributo `names` del índice a construir después de haber definido sus elementos generadores:

In [24]:
# Cambio de nombres en un índice multinivel desde su atributo.
df.index.names = ["nivel_1", "nivel_2"]
df

x1        x2
nivel_1 nivel_2                    
a       1        0.440159  0.362901
        2        0.303541  0.042880
b       1        0.525629  0.732826
        2        0.895001  0.755538

In [25]:
# Setting de nombres desde el constructor.
pd.MultiIndex.from_product([['a', 'b'], [1, 2]], names=["nivel_1", "nivel_2"])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['nivel_1', 'nivel_2'])

### Setting de índices multinivel para columnas.
En un DataFrame, las filas y columnas son elementos conceptualmente simétricos, y de la misma forma que las filas pueden tener índices de múltiples niveles, las columnas también pueden hacerlo. Consideremos el ejemplo siguiente, el cual replica una tabla resumen relativa a los indicadores geotécnicos de un conjunto de taludes en una operación minera:

In [26]:
# Índices y columnas multinivel
idx = pd.MultiIndex.from_product([[2018, 2019], ["sem_1", "sem_2"]], names=["año", "semestre"])
col = pd.MultiIndex.from_product(
    [["F10-B2445", "F10-B2680", "F11-B3210"], ["resist_corte", "prob_falla"]],
    names=["talud", "propiedad_geotecnica"]
)

In [27]:
# Generamos la data.
vals = np.array([
    [32.3, 8.82, 29.1, 8.90, 29.2, 8.05],
    [22.4, 10.12, 21.4, 10.20, 20.0, 10.92],
    [20.9, 11.22, 21.2, 11.03, 11.6, 22.92],
    [38.1, 7.56, 38.3, 7.02, 35.5, 6.78]
])

In [28]:
# Unimos todo en un DataFrame.
geot_data = pd.DataFrame(data=vals, index=idx, columns=col)

In [29]:
# Mostramos nuestro DataFrame en pantalla.
geot_data

talud                   F10-B2445               F10-B2680             \
propiedad_geotecnica resist_corte prob_falla resist_corte prob_falla   
año  semestre                                                          
2018 sem_1                   32.3       8.82         29.1       8.90   
     sem_2                   22.4      10.12         21.4      10.20   
2019 sem_1                   20.9      11.22         21.2      11.03   
     sem_2                   38.1       7.56         38.3       7.02   

talud                   F11-B3210             
propiedad_geotecnica resist_corte prob_falla  
año  semestre                                 
2018 sem_1                   29.2       8.05  
     sem_2                   20.0      10.92  
2019 sem_1                   11.6      22.92  
     sem_2                   35.5       6.78

Aquí vemos que el indexado múltiple tanto para filas y columnas puede ser de extrema utilidad. La anterior es, fundamentalmente, data 4D, donde las dimensiones corresponden al talud correspondiente, la propiedad geotécnica medida, el año y el semestre en el cual se miden estos valores. Con esto en consideración, podemos, por ejemplo, indexar la columna de nivel superior conforme el nombre de cada talud y obtener un DataFrame completo que contenga sólo la información de ese talud:

In [30]:
geot_data["F10-B2680"]

propiedad_geotecnica  resist_corte  prob_falla
año  semestre                                 
2018 sem_1                    29.1        8.90
     sem_2                    21.4       10.20
2019 sem_1                    21.2       11.03
     sem_2                    38.3        7.02

## Selección de datos en estructuras multinivel.
El indexado y slicing en una estructura de **Pandas** con índices múltiples está diseñado para ser intuitivo, y es muy útil si pensamos en los índices como dimensiones agregadas. Estudiaremos primero el indexado en series múltiples, para continuar luego con los DataFrames.

**a) Selección y slicing de data en series con multi-índices:** Consideremos la serie multinivel de alimentaciones a circuitos de molienda creada previamente:

In [31]:
# Nuestra serie multinivel.
feed_ser

SAG 1  2019    150350
       2020    156780
SAG 2  2019    149187
       2020    151223
SAG 3  2019    159003
       2020    157743
dtype: int64

Podemos acceder a valores únicos mediante la indexación de términos múltiples:

In [32]:
# Selección directa de un único dato.
feed_ser["SAG 2", 2020]

151223

Los índices múltiples también soportan indexaciones parciales (o indexar un único nivel de la serie). El resultado de esto es otra serie con los índices de menor nivel preservados:

In [33]:
feed_ser["SAG 2"]

2019    149187
2020    151223
dtype: int64

El *slicing parcial* también está disponible, siempre que el índice multinivel correspondiente esté ordenado:

In [34]:
# Los índices ya están ordenados, pero igualmente aplicamos un ordenamiento.
feed_ser.sort_index(ascending=True, inplace=True)

In [35]:
# Slicing parcial.
feed_ser["SAG 1":"SAG 2"]

SAG 1  2019    150350
       2020    156780
SAG 2  2019    149187
       2020    151223
dtype: int64

Con índices ordenados, se puede realizar cualquier indexación parcial en niveles inferiores pasando un slice vacío en el primer nivel del índice:

In [36]:
# Selección de segundo nivel.
feed_ser[:, 2019]

SAG 1    150350
SAG 2    149187
SAG 3    159003
dtype: int64

Otros tipos de indexación y selección (como los vistos en DataFrames con índices simples) también funcionan igual de bien. Por ejemplo, para el caso de la selección basada en masking, podemos tener:

In [37]:
# Masking.
feed_ser[feed_ser > 152000]

SAG 1  2020    156780
SAG 3  2019    159003
       2020    157743
dtype: int64

Asimismo, también es posible seleccionar datos mediante fancy indexing:

In [38]:
feed_ser[["SAG 1", "SAG 3"]]

SAG 1  2019    150350
       2020    156780
SAG 3  2019    159003
       2020    157743
dtype: int64

**b) Selección y slicing de data en DataFrames con multi-índices:** Un DataFrame con índices multinivel se comporta de la misma manera. Consideremos nuestro DataFrame `geot_data` construido previamente:

In [39]:
geot_data

talud                   F10-B2445               F10-B2680             \
propiedad_geotecnica resist_corte prob_falla resist_corte prob_falla   
año  semestre                                                          
2018 sem_1                   32.3       8.82         29.1       8.90   
     sem_2                   22.4      10.12         21.4      10.20   
2019 sem_1                   20.9      11.22         21.2      11.03   
     sem_2                   38.1       7.56         38.3       7.02   

talud                   F11-B3210             
propiedad_geotecnica resist_corte prob_falla  
año  semestre                                 
2018 sem_1                   29.2       8.05  
     sem_2                   20.0      10.92  
2019 sem_1                   11.6      22.92  
     sem_2                   35.5       6.78

Recordemos que las columnas corresponden al elemento de indexación primaria en un DataFrame, y que la sintaxis aplicada a series múltiplemente indexadas se aplica en este caso a los rótulos de columnas. Por ejemplo, podemos recuperar la data del talud `F11-B3210`, relativa a su probabilidad de falla (en %) con una operación muy sencilla:

In [40]:
geot_data["F11-B3210", "prob_falla"]

año   semestre
2018  sem_1        8.05
      sem_2       10.92
2019  sem_1       22.92
      sem_2        6.78
Name: (F11-B3210, prob_falla), dtype: float64

Es posible igualmente el hacer uso de métodos indexadores como `loc[]` y `iloc[]`:

In [41]:
geot_data.iloc[:2, :2]

talud                   F10-B2445           
propiedad_geotecnica resist_corte prob_falla
año  semestre                               
2018 sem_1                   32.3       8.82
     sem_2                   22.4      10.12

Estos indexadores proveen una vista de tipo arreglo de la data bidimensional subyacente, pero cada índice individual en `loc[]` y `iloc[]` puede ser pasado como una tupla de índices múltiples. Por ejemplo:

In [42]:
geot_data.loc[:, ("F10-B2680", "resist_corte")]

año   semestre
2018  sem_1       29.1
      sem_2       21.4
2019  sem_1       21.2
      sem_2       38.3
Name: (F10-B2680, resist_corte), dtype: float64

## Agregaciones en data multinivel.
Hemos visto previamente que **Pandas** nos provee con métodos nativos de agregación de datos, tales como `mean()`, `sum()` y `max()`. Para data multinivel, es posible replicar los métodos de cálculo aprendidos en términos de agregación mediante el control del parámetro `level`. Por ejemplo, retornemos a nuestro DataFrame `geot_data`. Quizás nos gustaría obtener la media de las mediciones por año y cada talud. Podemos hacer esto simplemente nombrando el nivel del índice respectivo que deseamos explorar (que, en este caso, es el año), usando para ello el método `groupby()`:

In [43]:
data_mean = geot_data.groupby(level='año').mean()
data_mean

talud                   F10-B2445               F10-B2680             \
propiedad_geotecnica resist_corte prob_falla resist_corte prob_falla   
año                                                                    
2018                        27.35       9.47        25.25      9.550   
2019                        29.50       9.39        29.75      9.025   

talud                   F11-B3210             
propiedad_geotecnica resist_corte prob_falla  
año                                           
2018                        24.60      9.485  
2019                        23.55     14.850

Haciendo uso del argumento `axis` en `groupby()`, podemos además calcular la media conforme de las niveles de las columnas:

In [44]:
data_mean.groupby(level="talud", axis=1).mean()

talud,F10-B2445,F10-B2680,F11-B3210
año,,,
2018,18.410,17.4000,17.0425
2019,19.445,19.3875,19.2000


## Comentarios finales.
La incorporación de series y DataFrames multinivel en nuestra caja de herramientas nos permite construir todo tipo de estructuras jerárquicamente indexadas en **Pandas**, preservando todos los esquemas de selección de datos que hemos aprendido para el caso de índices simples. En la próxima sección, revisaremos un tópico importante relativo al análisis de datos mediante este tipo de estructuras, y que guarda relación con la combinación de series y DataFrames. Si bien existen esquemas de combinación sencillos como los que aprendimos para la unión de arreglos de **Numpy**, **Pandas** nos ofrece opciones de ajuste de estas combinaciones que emulan mucho de lo que ocurre en la teoría de conjuntos. Puntualmente, veremos operaciones muy similares a la unión, intersección y diferencia simétrica de este tipo de objetos matemáticos, pero aplicados a series y DataFrames.